#Docker

#Django Backend Setup

In [ ]:
django-admin startproject myproject
cd myproject

In [ ]:
# myapp/models.py

from django.db import models

class Book(models.Model):
    title = models.CharField(max_length=100)
    author = models.CharField(max_length=100)

    def __str__(self):
        return self.title

In [ ]:
# myapp/views.py

from django.shortcuts import render, get_object_or_404, redirect
from .models import Book
from .forms import BookForm

def book_list(request):
    books = Book.objects.all()
    return render(request, 'book_list.html', {'books': books})

def book_detail(request, pk):
    book = get_object_or_404(Book, pk=pk)
    return render(request, 'book_detail.html', {'book': book})

def book_create(request):
    if request.method == 'POST':
        form = BookForm(request.POST)
        if form.is_valid():
            form.save()
            return redirect('book_list')
    else:
        form = BookForm()
    return render(request, 'book_form.html', {'form': form})

def book_update(request, pk):
    book = get_object_or_404(Book, pk=pk)
    if request.method == 'POST':
        form = BookForm(request.POST, instance=book)
        if form.is_valid():
            form.save()
            return redirect('book_list')
    else:
        form = BookForm(instance=book)
    return render(request, 'book_form.html', {'form': form})

def book_delete(request, pk):
    book = get_object_or_404(Book, pk=pk)
    if request.method == 'POST':
        book.delete()
        return redirect('book_list')
    return render(request, 'book_confirm_delete.html', {'book': book})

In [ ]:
# myapp/forms.py

from django import forms
from .models import Book

class BookForm(forms.ModelForm):
    class Meta:
        model = Book
        fields = ['title', 'author']

In [ ]:
# myproject/urls.py

from django.urls import path
from myapp import views

urlpatterns = [
    path('', views.book_list, name='book_list'),
    path('book/<int:pk>/', views.book_detail, name='book_detail'),
    path('book/new/', views.book_create, name='book_create'),
    path('book/<int:pk>/edit/', views.book_update, name='book_update'),
    path('book/<int:pk>/delete/', views.book_delete, name='book_delete'),
]

#React Frontend Setup

In [ ]:
npx create-react-app frontend
cd frontend

In [ ]:
# Using npm
npm install @mui/material @emotion/react @emotion/styled

# Using yarn
yarn add @mui/material @emotion/react @emotion/styled

In [ ]:
// src/components/BookList.js

import React, { useEffect, useState } from 'react';
import { List, ListItem, ListItemText, Button, Typography } from '@mui/material';

const BookList = ({ books }) => {
  return (
    <div>
      <Typography variant="h4" gutterBottom>
        Book List
      </Typography>
      <List>
        {books.map((book) => (
          <ListItem key={book.id}>
            <ListItemText primary={book.title} secondary={book.author} />
          </ListItem>
        ))}
      </List>
    </div>
  );
};

export default BookList;

In [ ]:
// src/components/BookForm.js

import React, { useState } from 'react';
import { TextField, Button, Typography, Grid } from '@mui/material';

const BookForm = ({ onSubmit }) => {
  const [title, setTitle] = useState('');
  const [author, setAuthor] = useState('');

  const handleSubmit = (e) => {
    e.preventDefault();
    onSubmit({ title, author });
    setTitle('');
    setAuthor('');
  };

  return (
    <div>
      <Typography variant="h4" gutterBottom>
        Add New Book
      </Typography>
      <form onSubmit={handleSubmit}>
        <Grid container spacing={2}>
          <Grid item xs={12}>
            <TextField
              required
              fullWidth
              label="Title"
              value={title}
              onChange={(e) => setTitle(e.target.value)}
            />
          </Grid>
          <Grid item xs={12}>
            <TextField
              required
              fullWidth
              label="Author"
              value={author}
              onChange={(e) => setAuthor(e.target.value)}
            />
          </Grid>
          <Grid item xs={12}>
            <Button type="submit" variant="contained" color="primary">
              Add Book
            </Button>
          </Grid>
        </Grid>
      </form>
    </div>
  );
};

export default BookForm;


In [ ]:
// src/App.js

import React, { useEffect, useState } from 'react';
import { Container, CssBaseline, Typography } from '@mui/material';
import BookList from './components/BookList';
import BookForm from './components/BookForm';

const App = () => {
  const [books, setBooks] = useState([]);

  useEffect(() => {
    // Fetch books from Django backend API
    fetch('http://localhost:8000/api/books/')
      .then((response) => response.json())
      .then((data) => setBooks(data))
      .catch((error) => console.error('Error fetching books:', error));
  }, []);

  const handleAddBook = (newBook) => {
    fetch('http://localhost:8000/api/books/', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify(newBook),
    })
      .then((response) => response.json())
      .then((data) => {
        setBooks([...books, data]);
      })
      .catch((error) => console.error('Error adding book:', error));
  };

  return (
    <Container maxWidth="sm">
      <CssBaseline />
      <Typography variant="h3" align="center" gutterBottom>
        Book Management App
      </Typography>
      <BookForm onSubmit={handleAddBook} />
      <BookList books={books} />
    </Container>
  );
};

export default App;

#ArangoDB Configuration

In [ ]:
# Dockerfile for ArangoDB

FROM arangodb/arangodb:latest

In [ ]:
# docker-compose.yml

version: '3'

services:
  django:
    build: ./myproject
    command: python manage.py runserver 0.0.0.0:8000
    volumes:
      - ./myproject:/app
    ports:
      - "8000:8000"
    depends_on:
      - arangodb

  frontend:
    build: ./frontend
    ports:
      - "3000:3000"
    volumes:
      - ./frontend:/app

  arangodb:
    build: ./arangodb
    environment:
      ARANGO_NO_AUTH: "1"
    ports:
      - "8529:8529"
    volumes:
      - arangodb_data:/var/lib/arangodb3

volumes:
  arangodb_data:

In [ ]:
docker-compose up -d

In [ ]:
docker-compose ps

In [ ]:
docker-compose down